In [84]:
using Revise
using Bilevel

using RigidBodyDynamics
using LinearAlgebra
using StaticArrays
using ForwardDiff
using DiffResults

using Bilevel: snopt, contact_τ, generate_autodiff_solver_fn, contact_jacobian!

In [9]:
urdf = joinpath("..", "urdf", "ball.urdf")
mechanism = parse_urdf(Float64, urdf)

floor = findbody(mechanism, "floor")
point = Point3D(default_frame(floor), SVector([0.,0.,0.]...))
normal = FreeVector3D(default_frame(floor), SVector([0.,0.,1.]...))
floor_obs = Obstacle(floor, point, normal, :xyz, 1.)

obstacles = [floor_obs]
env = Environment(mechanism, urdf, obstacles)
Δt = .005

0.005

In [10]:
sim_data = get_sim_data_direct(mechanism,env,Δt)

SimData(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., Environment(Contact[Contact(Spanning tree:
Vertex: world (root)
  Vertex: floor, Edge: floor_to_world
    Vertex: ball, Edge: floor_to_ball
No non-tree joints., ball, Point3D in "after_floor_to_ball": [0.0, 0.0, 0.0], Obstacle(floor, Point3D in "after_floor_to_world": [0.0, 0.0, 0.0], FreeVector3D in "after_floor_to_world": [0.0, 0.0, 1.0], [1.0 6.12323e-17 -1.0 -1.83697e-16; 0.0 1.0 1.22465e-16 -1.0; 0.0 0.0 0.0 0.0], 1.0, false))]), StateCache{…}(…), StateCache{…}(…), EnvironmentJacobianCache{…}(…), 0.005, Bilevel.VariableSelector(Dict(:qnext=>1:7,:vnext=>8:13), 13), Bilevel.ConstraintSelector(Dict(:dyn=>8:13,:kin=>1:7), [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13], Int64[], 13, 13, 0), :generate_solver_fn_sim_direct, :extract_sol_sim_direct, Bilevel.VariableSelector[], Bilevel.ConstraintSelector[], Dict[], Bilevel.VariableSelector[VariableSelect

In [81]:
function normal_τ!(τ,sim_data::SimData,Hi,envj::EnvironmentJacobian,dyn_bias,u0,v0,x_upper::AbstractArray{U},n::Int) where U
    num_contacts = length(sim_data.env.contacts)
    env = sim_data.env
    lower_vs = sim_data.normal_vs[n]
    lower_cs = sim_data.normal_cs[n]
    lower_options = sim_data.normal_options[n]
    h = sim_data.Δt
    
    ϕAs = []
    ϕbs = []
    for i = 1:num_contacts
        J = envj.contact_jacobians[i].J
        ϕ = envj.contact_jacobians[i].ϕ
        N = envj.contact_jacobians[i].N
    
        ϕA = h^2*N*Hi*J
        ϕb = N*(h^2*Hi*(dyn_bias - u0) .- h*v0) .- ϕ 

        push!(ϕAs,ϕA)
        push!(ϕbs,ϕb)
    end
    
    function eval_obj_(x::AbstractArray{L}) where L
        obj = 0.

        for i = 1:num_contacts
            c_n = lower_vs(x, Symbol("c_n", i))
            obj += c_n'*c_n
        end

        obj
    end

    function eval_cons_(x::AbstractArray{L}) where L
        g = []

        for i = 1:num_contacts
            c_n = lower_vs(x, Symbol("c_n", i))
            g = vcat(g, -c_n)
            g = vcat(g, ϕAs[i]*vcat(c_n, zeros(4)) + ϕbs[i])
        end
        
        g
    end

    fres = DiffResults.HessianResult(zeros(U, lower_vs.num_vars))
    gres = DiffResults.JacobianResult(zeros(U, lower_cs.num_cons), zeros(U, lower_vs.num_vars))
    solver_fn_ = generate_autodiff_solver_fn(eval_obj_,fres,eval_cons_,gres,lower_cs.eqs,lower_cs.ineqs)

    x0 = zeros(lower_vs.num_vars)

    xopt, info = Bilevel.auglag(solver_fn_, lower_cs.num_eqs, lower_cs.num_ineqs, x0, lower_options)
    
    τ .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
        contact_τ(cj, lower_vs(xopt, Symbol("c_n", i)), zeros(4))
    end
    
#     display("normal")
#     solver_fn_snopt = generate_autodiff_solver_fn(eval_obj_,eval_cons_,lower_cs.eqs,lower_cs.ineqs)
#     options_snopt = Dict{String, Any}()
#     options_snopt["Derivative option"] = 1
#     options_snopt["Verify level"] = -1 # -1 => 0ff, 0 => cheap
#     xopt_snopt, info_snopt = snopt(solver_fn_snopt, lower_cs.num_eqs, lower_cs.num_ineqs, x0, options_snopt)
#     display(info_snopt)
#     τ_snopt = zeros(length(τ))
#     τ_snopt .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
#         contact_τ(cj, lower_vs(xopt_snopt, Symbol("c_n", i)), zeros(4))
#     end
#     display("snopt")
#     display(xopt_snopt)
#     display(τ_snopt)
#     display("auglag")
#     display(xopt)
#     display(τ)
    
    xopt
end

function friction_τ!(τ,sim_data::SimData,Hi,envj::EnvironmentJacobian,dyn_bias,u0,v0,x_upper::AbstractArray{U},x_normal,n::Int) where U
    num_contacts = length(sim_data.env.contacts)
    env = sim_data.env
    normal_vs = sim_data.normal_vs[n]
    lower_vs = sim_data.fric_vs[n]
    lower_cs = sim_data.fric_cs[n]
    lower_options = sim_data.fric_options[n]
    h = sim_data.Δt
    
    Qds = []
    rds = []
    for i = 1:num_contacts
        J = envj.contact_jacobians[i].J
        ϕ = envj.contact_jacobians[i].ϕ
        N = envj.contact_jacobians[i].N
        
        Qd = h^2*J'*Hi*J
        rd = J'*(h^2*Hi*(dyn_bias - u0) .- h*v0)

        push!(Qds,Qd)
        push!(rds,rd)
    end

    function eval_obj_(x::AbstractArray{L}) where L
        obj = 0.

        for i = 1:num_contacts
            c_n = normal_vs(x_normal, Symbol("c_n", i))
            β = lower_vs(x, Symbol("β", i))
            z = vcat(c_n, β)
            obj += .5*z'*Qds[i]*z + z'*rds[i]
        end

        obj
    end

    function eval_cons_(x::AbstractArray{L}) where L
        g = []
        
        for i = 1:num_contacts
            c_n = normal_vs(x_normal, Symbol("c_n", i))
            β = lower_vs(x, Symbol("β", i))
            g = vcat(g, -β)
            g = vcat(g, sum(β) .- env.contacts[i].obstacle.μ * c_n)
        end
        
        g
    end

    fres = DiffResults.HessianResult(zeros(U, lower_vs.num_vars))
    gres = DiffResults.JacobianResult(zeros(U, lower_cs.num_cons), zeros(U, lower_vs.num_vars))
    solver_fn_ = generate_autodiff_solver_fn(eval_obj_,fres,eval_cons_,gres,lower_cs.eqs,lower_cs.ineqs)

    x0 = zeros(lower_vs.num_vars)

    xopt, info = Bilevel.auglag(solver_fn_, lower_cs.num_eqs, lower_cs.num_ineqs, x0, lower_options)
    
    τ .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
        contact_τ(cj, normal_vs(x_normal, Symbol("c_n", i)), lower_vs(xopt, Symbol("β", i)))
    end
    
#     display("friction")
#     solver_fn_snopt = generate_autodiff_solver_fn(eval_obj_,eval_cons_,lower_cs.eqs,lower_cs.ineqs)
#     options_snopt = Dict{String, Any}()
#     options_snopt["Derivative option"] = 1
#     options_snopt["Verify level"] = -1 # -1 => 0ff, 0 => cheap
#     xopt_snopt, info_snopt = snopt(solver_fn_snopt, lower_cs.num_eqs, lower_cs.num_ineqs, x0, options_snopt)
#     display(info_snopt)
#     τ_snopt = zeros(length(τ))
#     τ_snopt .= mapreduce(+, enumerate(envj.contact_jacobians)) do (i,cj)
#         contact_τ(cj, normal_vs(x_normal, Symbol("c_n", i)), lower_vs(xopt, Symbol("β", i)))
#     end
#     display("snopt")
#     display(xopt_snopt)
#     display(τ_snopt)
#     display("auglag")
#     display(xopt)
#     display(τ)

    xopt
end

function generate_solver_fn(sim_data,q0,v0,u0)
    x0 = sim_data.x0_cache[Float64]
    envj = sim_data.envj_cache[Float64]
    Δt = sim_data.Δt
    vs = sim_data.vs
    cs = sim_data.cs
    
    num_contacts = length(sim_data.env.contacts)
    num_vel = num_velocities(sim_data.mechanism)

    set_configuration!(x0, q0)
    set_velocity!(x0, v0)
    H = mass_matrix(x0)
    Hi = inv(H)
    
    contact_jacobian!(envj, x0)
    dyn_bias0 = dynamics_bias(x0) # TODO preallocate

    function eval_cons(x::AbstractArray{T}) where T
        xn = sim_data.xn_cache[T]
        
        normal_bias = Vector{T}(undef, num_vel)
        contact_bias = Vector{T}(undef, num_vel)
        g = Vector{T}(undef, cs.num_eqs + cs.num_ineqs) # TODO preallocate

        qnext = vs(x, :qnext)
        vnext = vs(x, :vnext)
        
        set_configuration!(xn, qnext)
        set_velocity!(xn, vnext)
        
        if (num_contacts > 0)
            # compute normal forces
            x_normal = normal_τ!(normal_bias, sim_data, Hi, envj, dyn_bias0, u0, v0, x, 1)
                        
            # compute friction forces
            friction_τ!(contact_bias, sim_data, Hi, envj, dyn_bias0, u0, v0, x, x_normal, 1)
        end
        config_derivative = configuration_derivative(xn)
        dyn_bias = dynamics_bias(xn)

        g[cs(:kin)] .= qnext .- q0 .- Δt .* config_derivative
        g[cs(:dyn)] .= H * (vnext - v0) .- Δt .* (u0 .- dyn_bias .- contact_bias)

        g
    end
        
    eval_cons
end

generate_solver_fn (generic function with 1 method)

In [90]:
q0 = [1., 0., 0., 0., 0., 0., 1.]
v0 = [0., 0., 0., 1., 0., 0.]
u0 = [0., 0., 0., 0., 0., 0.]

sim_data.normal_options[1]["num_fosteps"] = 0
sim_data.normal_options[1]["num_sosteps"] = 15
sim_data.normal_options[1]["c"] = 1
sim_data.normal_options[1]["c_fos"] = 1
sim_data.normal_options[1]["c_sos"] = 1

sim_data.fric_options[1]["num_fosteps"] = 0
sim_data.fric_options[1]["num_sosteps"] = 15
sim_data.fric_options[1]["c"] = 1
sim_data.fric_options[1]["c_fos"] = 1
sim_data.fric_options[1]["c_sos"] = 1

eval_cons = generate_solver_fn(sim_data,q0,v0,u0)

eval_cons (generic function with 1 method)

In [91]:
x = vcat(q0,v0)
sol = eval_cons(x)

13-element Array{Float64,1}:
  0.0                  
  0.0                  
  0.0                  
  0.0                  
 -0.005                
  0.0                  
  0.0                  
  0.0                  
  0.0                  
  0.0                  
 -3.429977435310207e-11
 -7.54588907562469e-15 
  0.04904999999999349  

In [92]:
J = ForwardDiff.jacobian(eval_cons, x)

13×13 Array{Float64,2}:
  1.0     0.0      0.0      0.0   0.0  …   0.0      0.0     0.0     0.0  
  0.0     1.0      0.0      0.0   0.0      0.0      0.0     0.0     0.0  
  0.0     0.0      1.0      0.0   0.0      0.0      0.0     0.0     0.0  
  0.0     0.0      0.0      1.0   0.0     -0.0025   0.0     0.0     0.0  
 -0.01    0.0      0.0      0.0   1.0      0.0     -0.005   0.0     0.0  
  0.0     0.0      0.0     -0.01  0.0  …   0.0      0.0    -0.005   0.0  
  0.0     0.0      0.01     0.0   0.0      0.0      0.0     0.0    -0.005
  0.0     0.0      0.0      0.0   0.0      0.0      0.0     0.0     0.0  
  0.0     0.0      0.0      0.0   0.0      0.0      0.0     0.0     0.0  
  0.0     0.0      0.0      0.0   0.0      1.0e-6   0.0     0.0     0.0  
  0.0     0.0     -0.0981   0.0   0.0  …   0.0      1.0     0.0     0.0  
  0.0     0.0981   0.0      0.0   0.0      0.005    0.0     1.0     0.0  
  0.0981  0.0      0.0      0.0   0.0      0.0      0.0     0.0     1.0  

In [93]:
ϵ = sqrt(eps(1.))
J_num = zeros(size(J))
for i = 1:length(x)
    δ = zeros(length(x))
    δ[i] = ϵ
    J_num[:,i] = (eval_cons(x + δ) .- sol)/ϵ
end
J_num

13×13 Array{Float64,2}:
  1.0      0.0           0.0          …   0.0      0.0     0.0     0.0  
  0.0      1.0           0.0              0.0      0.0     0.0     0.0  
  0.0      0.0           1.0              0.0      0.0     0.0     0.0  
  0.0      0.0           0.0             -0.0025   0.0     0.0     0.0  
 -0.01    -5.82077e-11   5.82077e-11      0.0     -0.005   0.0     0.0  
  0.0      0.0           0.0          …   0.0      0.0    -0.005   0.0  
  0.0      0.0           0.01             0.0      0.0     0.0    -0.005
  0.0      0.0           0.0              0.0      0.0     0.0     0.0  
  0.0      0.0           0.0              0.0      0.0     0.0     0.0  
  0.0      0.0           0.0              1.0e-6   0.0     0.0     0.0  
  0.0      0.0          -0.0981       …   0.0      1.0     0.0     0.0  
  0.0      0.0981        0.0              0.005    0.0     1.0     0.0  
  0.0981  -9.31323e-10  -9.31323e-10      0.0      0.0     0.0     1.0  

In [94]:
maximum(abs.(J_num - J))

9.313225746154785e-10